# Chapter 11: Dense Vector Representations
## Cooccurrence Matrices and Mutual Information
We compute word embeddings from cooccurrence matrices

Programs from the book: [_Python for Natural Language Processing_](https://link.springer.com/book/9783031575488)

__Author__: Pierre Nugues

## Modules

In [1]:
import torch
import torch.nn as nn
import os
import regex as re
from collections import Counter
import collections
from math import log
import numpy as np
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import fbpca
from sklearn.decomposition import PCA, TruncatedSVD

## Constants

In [2]:
K = 4  # context window
alpha = torch.tensor(0.75)

torch.manual_seed(1234)

## Corpus Files

We read the files and we store the corpus in a string

In [3]:
PATH = '../datasets/'

In [4]:
CORPUS = 'HOMER'  # 'DICKENS'

In [5]:
if CORPUS == 'DICKENS':
    folder = PATH + 'dickens/'
elif CORPUS == 'HOMER':
    folder = PATH + 'classics/'

In [6]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [7]:
if CORPUS == 'DICKENS':
    files = get_files(folder, 'txt')
elif CORPUS == 'HOMER':
    files = ['iliad.txt', 'odyssey.txt']
files

['iliad.txt', 'odyssey.txt']

In [8]:
files = [folder + file for file in files]
files

['../datasets/classics/iliad.txt', '../datasets/classics/odyssey.txt']

In [9]:
text = ''
for file in files:
    with open(file, encoding='utf8') as f:
        text += ' ' + f.read().strip()

In [10]:
text[:100]

' BOOK I\n\nSing, O goddess, the anger of Achilles son of Peleus, that brought\ncountless ills upon the '

## Tokenizing

In [11]:
# text = open('corpus.txt', encoding='utf8').read()
text = text.lower()
words = re.findall(r'\p{L}+', text)

In [12]:
vocab = sorted(list(set(words)))
vocab_size = len(vocab)
idx2word = dict(enumerate(vocab))
word2idx = {v: k for k, v in idx2word.items()}

In [13]:
vocab[:10]

['a',
 'abantes',
 'abarbarea',
 'abas',
 'abate',
 'abated',
 'abetting',
 'abhorred',
 'abians',
 'abide']

In [14]:
idx2word

{0: 'a',
 1: 'abantes',
 2: 'abarbarea',
 3: 'abas',
 4: 'abate',
 5: 'abated',
 6: 'abetting',
 7: 'abhorred',
 8: 'abians',
 9: 'abide',
 10: 'abides',
 11: 'abiding',
 12: 'ablaze',
 13: 'able',
 14: 'ablerus',
 15: 'aboard',
 16: 'abode',
 17: 'abodes',
 18: 'abominable',
 19: 'abominably',
 20: 'abounding',
 21: 'abounds',
 22: 'about',
 23: 'above',
 24: 'abroad',
 25: 'absence',
 26: 'absent',
 27: 'absolutely',
 28: 'absurd',
 29: 'abundance',
 30: 'abundant',
 31: 'abundantly',
 32: 'abuse',
 33: 'abusing',
 34: 'abydos',
 35: 'acamas',
 36: 'acastus',
 37: 'accept',
 38: 'acceptable',
 39: 'acceptance',
 40: 'accepted',
 41: 'accident',
 42: 'accompanied',
 43: 'accompaniment',
 44: 'accomplish',
 45: 'accomplished',
 46: 'accomplishment',
 47: 'accomplishments',
 48: 'accord',
 49: 'according',
 50: 'accordingly',
 51: 'account',
 52: 'accounted',
 53: 'accounting',
 54: 'accounts',
 55: 'accurately',
 56: 'accursed',
 57: 'accusing',
 58: 'accustomed',
 59: 'acessamenus',
 

In [15]:
words_idx = [word2idx[word] for word in words]

In [16]:
words_idx[:5]

[1043, 4355, 7663, 5691, 3697]

## Cooccurrences

In [17]:
def update_counts(start_dict: dict[int, float],
                  lc: list[int],
                  rc: list[int]) -> dict[int, float]:
    for i, word in enumerate(rc, start=1):
        if word in start_dict:
            start_dict[word] += 1.0/i
        else:
            start_dict[word] = 1.0/i
    for i, word in enumerate(lc[::-1], start=1):
        if word in start_dict:
            start_dict[word] += 1.0/i
        else:
            start_dict[word] = 1.0/i
    return start_dict

In [18]:
def build_C(words_idx: list[int],
            K: int) -> dict[int, dict[int, float]]:
    C_dict = dict()
    for i, word in tqdm(enumerate(words_idx)):
        if word not in C_dict:
            C_dict[word] = dict()
        lc = words_idx[i - K:i]
        rc = words_idx[i + 1:i + K + 1]
        C_dict[word] = update_counts(C_dict[word],
                                     lc, rc)
    return C_dict

In [19]:
C_dict = build_C(words_idx, K)

271506it [00:00, 588969.65it/s]


In [20]:
C_dict[10]

{575: 1.0,
 8548: 1.8333333333333333,
 7552: 0.3333333333333333,
 4432: 1.25,
 5675: 1.0,
 3991: 0.5,
 5278: 0.3333333333333333,
 5735: 0.5833333333333333,
 1703: 0.5,
 0: 0.25,
 9493: 1.0,
 5461: 0.5,
 9111: 0.25,
 2960: 0.5,
 8309: 0.3333333333333333,
 352: 0.25,
 4603: 1.0,
 1278: 0.5,
 3075: 0.3333333333333333,
 1333: 0.25}

## PCA

In [21]:
def build_matrix(C_dict):
    cooc_mat = np.zeros((len(C_dict), len(C_dict)))
    for k, coocs in C_dict.items():
        for c, cnt in coocs.items():
            cooc_mat[k, c] = float(cnt)
    return cooc_mat

In [22]:
cooc_mat = build_matrix(C_dict)

In [23]:
(U, s, Va) = fbpca.pca(cooc_mat, 50)

In [24]:
import pandas as pd

df = pd.DataFrame(U @ np.diag(s),
                  index=[idx2word[i] for i in range(len(idx2word))])

In [25]:
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
a,994.866729,-372.829308,-264.771206,74.316433,94.715923,32.390645,38.368668,73.729283,83.784158,-231.234859,...,11.438571,7.077939,-1.362754,2.992231,-9.522612,8.478756,7.175983,-3.912524,-3.934935,11.462798
abantes,-9.725727,0.506849,1.916567,-0.834800,0.130849,0.859824,-0.057988,0.626025,0.361392,0.143322,...,0.024086,-0.130608,0.130688,0.132761,0.080599,-0.194315,-0.069660,0.237923,-0.183604,-0.049493
abarbarea,-11.034901,-0.433111,1.337218,-0.427344,0.043983,0.828580,0.268999,0.570264,0.434622,0.333783,...,-0.021388,-0.058001,0.185364,-0.157143,0.131654,0.049328,0.120200,0.137766,0.024228,-0.026636
abas,-10.431065,-1.577821,1.780151,-0.271763,-0.093521,0.412158,0.188440,-0.120887,0.272247,-0.120940,...,0.004315,-0.007496,0.080494,-0.088969,0.111484,0.068994,0.063156,0.060472,-0.064561,0.012620
abate,-11.151531,-0.589540,0.848519,-0.482948,-0.197600,0.708296,0.505059,0.541680,0.190544,0.014798,...,-0.252218,-0.098630,0.008405,-0.228302,0.121642,0.017648,0.029923,0.294673,-0.031513,0.048705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zeal,-10.478428,-0.649052,1.208903,-0.379662,0.472562,0.838413,-0.273628,0.792311,0.567987,-0.534318,...,0.143022,-0.015247,0.472579,-0.315559,0.102347,0.170921,-0.050042,0.037291,-0.189993,0.193479
zelea,-10.159126,-1.661208,1.416362,-1.604377,-0.470596,0.383599,0.872784,-0.055871,0.182173,-0.378975,...,-0.040561,0.005697,0.105649,-0.208498,0.238788,-0.104536,-0.096222,0.171866,-0.128647,0.110083
zephyrus,-9.974867,-1.133103,0.714964,0.500799,0.123526,0.802366,0.393719,0.039474,0.105058,0.413836,...,-0.493320,0.107333,-0.139328,0.016145,-0.141045,0.036691,0.136955,0.332895,-0.182573,0.231168
zethus,-10.662615,-1.060462,1.457813,0.052777,-0.028716,0.310988,0.207314,0.112787,0.442833,-0.138740,...,0.046242,-0.085706,-0.151597,-0.036831,-0.045117,0.068507,0.126475,-0.044864,0.065818,0.170407


In [26]:
df.to_csv('cooc50d.txt', sep=' ', header=False)

## Similarity

A few test words

In [27]:
if CORPUS == 'HOMER':
    test_words = ['he', 'she', 'ulysses', 'penelope', 'achaeans', 'trojans',
                  'achilles', 'sea', 'helen', 'ship', 'her', 'fight']
elif CORPUS == 'DICKENS':
    test_words = ['he', 'she', 'her', 'sea', 'ship',
                  'fight', 'table', 'london', 'monday']

In [28]:
def most_sim_vecs(u, E, N=10):
    cos = nn.CosineSimilarity()
    cos_sim = cos(u.unsqueeze(dim=0), E)
    sorted_vectors = sorted(range(len(cos_sim)),
                            key=lambda k: -cos_sim[k])
    return sorted_vectors[1:N + 1]

In [29]:
def sim_test_words(test_words, word2idx, U, N=10):
    most_sim_words = {}
    if type(U) == np.ndarray:
        E = torch.from_numpy(U)
    for w in test_words:
        most_sim_words[w] = most_sim_vecs(E[word2idx[w]], E, N)
        most_sim_words[w] = list(map(idx2word.get, most_sim_words[w]))
        print(w, most_sim_words[w])

Original

In [30]:
sim_test_words(test_words, word2idx, cooc_mat, N=3)

he ['she', 'it', 'they']
she ['he', 'they', 'ulysses']
ulysses ['achilles', 'hector', 'telemachus']
penelope ['telemachus', 'ulysses', 'juno']
achaeans ['danaans', 'argives', 'trojans']
trojans ['achaeans', 'danaans', 'argives']
achilles ['hector', 'ulysses', 'patroclus']
sea ['trojans', 'ground', 'town']
helen ['jove', 'them', 'hector']
ship ['on', 'fire', 'in']
her ['him', 'them', 'his']
fight ['first', 'on', 'town']


Reduced

In [31]:
sim_test_words(test_words, word2idx, U @ np.diag(s), N=3)

he ['she', 'it', 'ulysses']
she ['he', 'minerva', 'ulysses']
ulysses ['hector', 'menelaus', 'achilles']
penelope ['telemachus', 'neptune', 'ulysses']
achaeans ['danaans', 'argives', 'suitors']
trojans ['sea', 'danaans', 'achaeans']
achilles ['hector', 'ulysses', 'agamemnon']
sea ['trojans', 'earth', 'town']
helen ['juno', 'alexandrus', 'jove']
ship ['fire', 'on', 'way']
her ['him', 'them', 'minerva']
fight ['first', 'face', 'fall']


In [32]:
sim_test_words(test_words, word2idx, U, N=3)

he ['patroclus', 'apollo', 'stranger']
she ['handmaids', 'maids', 'maidens']
ulysses ['antinous', 'telemachus', 'eumaeus']
penelope ['euryclea', 'telemachus', 'stranger']
achaeans ['trojans', 'argives', 'suitors']
trojans ['achaeans', 'immortals', 'gates']
achilles ['ajax', 'diomed', 'sarpedon']
sea ['door', 'waves', 'gates']
helen ['sake', 'joy', 'evermore']
ship ['crew', 'raft', 'heifer']
her ['maids', 'room', 'bosom']
fight ['sacrifice', 'return', 'sack']


## Mutual Information

In [33]:
def build_pmi_mat(cooc_mat):
    pair_cnt = np.sum(cooc_mat)
    wi_rel_freq = np.sum(cooc_mat, axis=1) / pair_cnt
    wc_rel_freq = np.sum(cooc_mat, axis=0) / pair_cnt

    cooc_mat /= pair_cnt
    cooc_mat /= wc_rel_freq  # Divide by rows
    cooc_mat = (cooc_mat.T / wi_rel_freq).T  # Divide by cols

    mi_mat = np.log2(cooc_mat,
                     out=np.zeros_like(cooc_mat),
                     where=(cooc_mat != 0))
    mi_mat[mi_mat < .0] = 0.0
    return mi_mat

In [34]:
def build_pmi_mat(cooc_mat):
    pair_cnt = np.sum(cooc_mat)
    wi_rel_freq = np.sum(cooc_mat, axis=1) / pair_cnt
    wc_rel_freq = np.sum(cooc_mat, axis=0) / pair_cnt

    mi_mat = cooc_mat / pair_cnt  # P(w_i, w_j)
    mi_mat /= wc_rel_freq  # Divide by rows P(w_i, w_j)/P(w_j)
    mi_mat = (mi_mat.T / wi_rel_freq).T  # Divide by cols

    mi_mat = np.log2(mi_mat,
                     out=np.zeros_like(mi_mat),
                     where=(mi_mat != 0))
    mi_mat[mi_mat < 0.0] = 0.0
    return mi_mat

In [35]:
cooc_mat

array([[110.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [36]:
pmi_mat = build_pmi_mat(cooc_mat)

In [37]:
pmi_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
(U_pmi, s_pmi, Va_pmi) = fbpca.pca(pmi_mat, 50)

## Similarities

Original

In [39]:
sim_test_words(test_words, word2idx, pmi_mat, N=3)

he ['his', 'was', 'as']
she ['her', 'herself', 'said']
ulysses ['said', 'telemachus', 'answered']
penelope ['eurymachus', 'iphthime', 'misconduct']
achaeans ['trojans', 'the', 'they']
trojans ['achaeans', 'the', 'danaans']
achilles ['him', 'hector', 'patroclus']
sea ['shore', 'waves', 'the']
helen ['cisseus', 'polycaste', 'sake']
ship ['fibre', 'sails', 'vessel']
her ['she', 'and', 'room']
fight ['battle', 'fought', 'raged']


Reduced

In [40]:
sim_test_words(test_words, word2idx, U_pmi @ np.diag(s_pmi), N=3)

he ['him', 'it', 'and']
she ['her', 'herself', 'wept']
ulysses ['telemachus', 'said', 'this']
penelope ['replied', 'child', 'queen']
achaeans ['trojans', 'danaans', 'fight']
trojans ['achaeans', 'danaans', 'fight']
achilles ['patroclus', 'him', 'then']
sea ['waves', 'shore', 'deep']
helen ['lovely', 'diana', 'venus']
ship ['inside', 'our', 'way']
her ['she', 'herself', 'room']
fight ['fighting', 'battle', 'trojans']


In [41]:
sim_test_words(test_words, word2idx, U_pmi, N=3)

he ['him', 'it', 'necessary']
she ['moaned', 'herself', 'impudent']
ulysses ['eurymachus', 'stranger', 'telemachus']
penelope ['icarius', 'replied', 'queen']
achaeans ['trojans', 'danaans', 'fight']
trojans ['danaans', 'achaeans', 'lycians']
achilles ['patroclus', 'vainly', 'him']
sea ['surf', 'waves', 'shore']
helen ['marpessa', 'lovely', 'wife']
ship ['mast', 'sand', 'asleep']
her ['continuing', 'nestlings', 'undoing']
fight ['fighting', 'battle', 'trojans']


In [42]:
df = pd.DataFrame(U_pmi @ np.diag(s_pmi),
                  index=[idx2word[i] for i in range(len(idx2word))])

In [43]:
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
a,41.911392,15.841310,26.293751,10.871785,-13.767154,22.264312,-21.898471,-11.405866,1.050081,-17.722387,...,1.897705,-0.752465,5.709967,-5.431149,1.898075,-3.962692,0.282949,-0.395725,-1.332035,-3.112229
abantes,-0.662127,1.585866,-0.258516,-0.042244,0.880094,0.135028,0.176419,0.666590,0.285193,0.802858,...,0.793079,-1.911862,0.524281,-0.216087,-0.609036,-0.387335,1.005242,-1.291932,1.305416,0.899632
abarbarea,-1.481659,0.076777,0.574981,-2.403991,-0.452189,0.577659,0.173647,0.765485,0.367199,0.012344,...,0.427744,-0.003841,0.559767,-0.455184,-0.826249,0.247284,0.128037,1.024146,0.275656,-1.187099
abas,-1.904638,0.814443,1.431647,-1.911510,0.338383,0.157091,0.527633,0.143973,-1.310026,0.363041,...,-0.571924,-0.106950,-0.387943,0.007158,0.099541,-0.459429,-0.157860,0.403389,0.035745,-0.115201
abate,-1.711426,-0.643791,0.685235,0.219523,-0.105462,-0.726950,-0.760463,0.797602,0.339476,-0.624958,...,-1.077998,-0.735651,-0.407523,0.207979,-0.847255,0.202912,-0.008020,0.722968,-0.564704,0.713473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zeal,-1.381949,0.501979,0.403622,-1.450830,0.445523,-0.740679,0.569121,-1.462584,0.297887,0.678942,...,-0.353196,-0.115065,-0.706534,0.283219,0.088534,0.251131,0.486820,0.347653,-0.784174,1.033128
zelea,-1.676605,-0.493245,1.151352,-0.076168,-0.181442,-0.020254,0.068226,-0.752275,-0.971610,-0.123758,...,-0.158039,-0.560742,0.040674,0.066569,0.190625,-0.152878,1.214323,0.287296,-0.154206,0.134780
zephyrus,-0.481868,0.134501,0.019616,0.118269,0.419733,0.327799,0.005745,1.696392,0.661736,0.947663,...,-0.271718,-1.211701,-0.090945,-0.853355,-0.313386,-1.239951,-0.836818,-0.984263,0.461912,1.013027
zethus,-1.127127,0.243547,0.294333,-3.247071,0.795919,1.760022,0.833923,-0.627370,0.614347,1.284684,...,-0.422945,1.162998,1.008782,-1.121522,0.599425,-1.459280,0.041844,0.394864,-1.103873,0.706973
